<a href="https://colab.research.google.com/github/techie-coder/MindWell/blob/main/MoodAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq langchain langchain-core langchain_groq

import json

from google.colab import userdata
api_key = userdata.get('groq-api-key')

from langchain.chains import LLMChain
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)


from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_groq import ChatGroq


groq_api_key = api_key
model = 'llama3-8b-8192'

"""
    This function is the main entry point of the application. It sets up the Groq client, the Streamlit interface, and handles the chat interaction.
    """


# Initialize Groq Langchain chat object and conversation
groq_chat = ChatGroq(
            groq_api_key=groq_api_key,
            model_name=model
    )

conversational_memory_length = 5 #number of previous messages the chatbot will remember during the conversation

memory = ConversationBufferWindowMemory(k=conversational_memory_length, memory_key="chat_history", return_messages=True)



In [10]:
def cbt(user_question):


    print("Entering cbt...")

    system_prompt = """You are MindWell, a chatbot designed to generate congnitive restructured sentences in reponse to a context(primarily negative) provided my the university students who are suffering from mental health problems. Make sure to follow these guidelines:

    1. Always be supportive and postive

    2. Never be judgemental

    3. Make the student feel safe and maintain anonymity

    4. Don't apologise to him/her for his/her situation
    """



    if user_question == None or user_question == "":
        return

    if user_question:

    # Construct a chat prompt template using various components
      prompt = ChatPromptTemplate.from_messages(
            [
              SystemMessage(
                  content=system_prompt
                  ),  # This is the persistent system prompt that is always included at the start of the chat.

              MessagesPlaceholder(
                  variable_name="chat_history"
                  ),  # This placeholder will be replaced by the actual chat history during the conversation. It helps in maintaining context.

              HumanMessagePromptTemplate.from_template(
                  "{human_input}"
                  ),  # This template is where the user's current input will be injected into the prompt.
                  ]
              )
      # Create a conversation chain using the LangChain LLM (Language Learning Model)
      conversation = LLMChain(
            llm=groq_chat,  # The Groq LangChain chat object initialized earlier.
            prompt=prompt,  # The constructed prompt template.
            verbose=False,   # TRUE Enables verbose output, which can be useful for debugging.
            memory=memory,  # The conversational memory object that stores and manages the conversation history.
              )
            # The chatbot's answer is generated by sending the full prompt to the Groq API.
      response = conversation.predict(human_input=user_question)
      #print("Chatbot:", response)
      return json.loads(response)["answer"]


In [ ]:
from textblob import TextBlob
import numpy as np

def analyze_sentiment(text):
    """
    Analyze the sentiment of the given text.
    Returns a value between -1 (very negative) and 1 (very positive).
    """
    return TextBlob(text).sentiment.polarity


def check_mood_trend(sentiment_history, threshold=-0.3):
    """
    Check if there's a significant negative trend in mood.
    Returns True if the trend is concerning, False otherwise.
    """
    if len(sentiment_history) < 3:
        return False

    recent_avg = np.mean(sentiment_history[-3:])
    overall_avg = np.mean(sentiment_history)
    return recent_avg < overall_avg and recent_avg < threshold



def analyse_mood():
    """
    This function is the main entry point of the application. It sets up the Groq client,
    the chat interface, and handles the chat interaction with mood analysis.
    """

    print("Entered mood_analysis")

    print("Hey, how are you feeling today? It's okay to share whatever's on your mind—I'm here to listen!")

    # Customize the system prompt to include mood analysis
    system_prompt = """
   You are an empathetic and supportive chatbot named MindWell, designed to interact with university students. Your primary goals are to engage in conversation, analyze the student's mood and provide necessary helpful information if needed. Follow these guidelines:

    1. Mood Analysis:
       - Pay close attention to language indicative of stress, anxiety, excitement, frustration, or other emotions common in university life.
       - Consider context such as exam periods, start of semester, graduation, etc.
       - Look for signs of homesickness, academic pressure, social challenges, or career concerns.

    2. Response Approach:
       - Adjust your tone to match the student's mood - be upbeat for positive moods, supportive for negative ones.
       - Use language and references familiar to university students.
       - Be encouraging and motivational, especially when detecting stress or anxiety.

    3. Topic Awareness:
       - Be prepared to discuss common university topics: classes, exams, assignments, campus life, extracurricular activities, internships, and career planning.
       - Offer relevant advice or resources when appropriate.

    4. Mood Reflection:
       - Subtly reflect your mood analysis in your responses without explicitly stating it.
       - Use phrases like "It sounds like you might be feeling..." or "That must be..." to show empathy and understanding.

    5. Support and Resources:
       - If you detect signs of serious distress or consistent negative moods, gently suggest professional campus resources like counseling services.
       - Promote healthy habits and stress-management techniques relevant to student life.

    6. Mood Tracking:
       - Be aware that the user's mood and sentiment are being tracked over time.
       - If you're informed of significant negative trends or high mood variations, incorporate this knowledge into your responses subtly.
       - Never directly mention the tracking or specific scores.

    7. If the user mentions sensitive topics like inappropriate relationships, underage interactions, or illegal activities, acknowledge the complexity but avoid delving deeper into the subject. Gently redirect the conversation while offering general emotional support.

    8. If the conversation reveals an inappropriate or illegal age dynamic, remain neutral and end the conversation on that topic. Provide general support and resources for the user to seek help in a safe and ethical manner.

    9. When handling sensitive situations, offer emotional validation but suggest professional help. If necessary, provide resources such as helplines or mental health services while avoiding giving advice on legal or ethical issues.

    10. Ensure that the chatbot does not collect or store sensitive personal information about the user’s relationships, activities, or age. Prioritize the user's privacy while encouraging safe, responsible behavior.

    11. When the conversation involves sensitive legal or ethical boundaries, conclude the interaction gently but firmly. Express concern for the user's well-being and recommend seeking help elsewhere.

    12. In cases where the conversation suggests imminent harm or illegal activity, refer the user to appropriate authorities or helplines. Do not advise the user directly on what actions to take.

    13. When the user discusses complex emotions or sensitive relationships, use reflective questions to allow the user to process their emotions without encouraging further discussion of the sensitive topic.

    14. Refrain from saying that you cannot continue the conversation, instead say that you would like the user to connect to a professional who can help him/her regarding the situation.

    Always maintain a friendly, non-judgmental, and supportive tone throughout the conversation.
    """

    sentiment_history = []

    user_chat=[]

    while True:
        user_question = input("You: ")

        if user_question == None or user_question == "":
            break

        if user_question:

          a = analyze_sentiment(user_question)
          sentiment_history.append(a)
          print("Sentiment:", a)

          if len(sentiment_history) > 2:
            sentiment_history.pop(0)

          mood_variation = np.std(sentiment_history) if len(sentiment_history) > 1 else 0
          print("Mood variation:", mood_variation)


          # Prepare additional context for the chatbot
          mood_context = ""

          if mood_variation > 0.1:
                print(cbt(user_question))
                continue

          if mood_variation > 0.05:
                mood_context = "The user's mood seems to have been fluctuating significantly and trending negative recently. Be extra supportive and consider gently suggesting university counseling services if appropriate."


          prompt = ChatPromptTemplate.from_messages([
                SystemMessagePromptTemplate.from_template(system_prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                SystemMessagePromptTemplate.from_template(mood_context),
                HumanMessagePromptTemplate.from_template("{human_input}")
            ])

          conversation = LLMChain(
                llm=groq_chat,
                prompt=prompt,
                verbose=False,
                memory=memory,
            )

          response = conversation.predict(human_input=user_question)
          print(response)

          user_chat.append(user_question)



if __name__ == "__main__":
    analyse_mood()